In [21]:
import json
import pandas as pd
import numpy as np

In [22]:
# Read Summarisation Model results from json file.

with open("../data/Model_Results/summarisation_results.jsonl", 'r') as json_file:
    json_list = list(json_file)

    
summarisationResults = {}
for json_str in json_list:
    
    result = json.loads(json_str)
    summaryList = sorted(result["sentence_scores"], key=lambda x:x[1], reverse=True)[:4]
    summary = ""
    for sentence in summaryList:
        summary += sentence[0] + ' '
    
    summarisationResults[result["id"]] = summary

In [23]:
summarisationData = pd.DataFrame(data=summarisationResults, index=[0])

In [24]:
display(summarisationData)

,11972.json,11685.json,11096.json,5209.json,9524.json,5962.json,7070.json,1046.json,12849.json,13270.json,...,11576.json,3461.json,9464.json,10227.json,11707.json,3425.json,2977.json,294.json,3580.json,8384.json
0,"""Building a wall"" on the border ""will take lit...","Those numbers show that as of October 2015, th...","If he did, he would have known that Senator Mc...","""…She supports taking $500 billion away from M...","Scott Walker helped run a ""criminal scheme"" to...",The campaign accurately quoted a figure whose ...,"So $30-31 million per year would, in fact, be ...",The Obama administration has emphasized many o...,"""It matters who’s leading the country, and it ...","Pence described the donors as major. ""The nati...",...,We’re not sure Sanders made that entirely clea...,"The $20 million designated for Cuba ""focuses o...",There also are no Asian or Pacific Islander Re...,"""The United States is in the longest stretch o...","""Secretary Clinton changes her position on thi...","Under the header ""New jobs created by the Stre...","They said low pay, increased work demands and ...","• Comstock, an adviser and frequent spokeswoma...","""House Republicans under Paul Ryan's leadershi...","""I will work in a bipartisan way to get it don..."


In [25]:
# Read DeClarE results from json file.
with open('../data/Model_Results/DeClarE_Results.json') as file:
    declareResults = json.load(file)

In [26]:
exist = 0
nExist = 0
for json_id in summarisationData.columns:

    if json_id not in declareResults.keys():
        del summarisationData[json_id]

In [27]:
display(summarisationData)

,10540.json,4148.json,8705.json,10683.json,7057.json,9727.json,1926.json,11418.json,3652.json,8187.json,...,6411.json,361.json,11576.json,3461.json,10227.json,11707.json,3425.json,2977.json,294.json,3580.json
0,"""When did the decline of coal start? ""That sta...",Sanders ended the tweet with his signature has...,The law added about3.1 million young adults to...,"• ""63 percent of the American people chose not...",That would be speculation -- but that speculat...,"""The president's come out with rules that say ...",And in a July 2009 speech at the Council on Fo...,"""African-American youth unemployment is 51 per...",He took exactly none of his own deficit reduct...,"""He’s the person who invented the scheme to re...",...,"""As president, Barack Obama has never visited ...",Asked during a Texas campaign event about the ...,We’re not sure Sanders made that entirely clea...,"The $20 million designated for Cuba ""focuses o...","""The United States is in the longest stretch o...","""Secretary Clinton changes her position on thi...","Under the header ""New jobs created by the Stre...","They said low pay, increased work demands and ...","• Comstock, an adviser and frequent spokeswoma...","""House Republicans under Paul Ryan's leadershi..."


In [28]:
# Read Justification data from tsv into dataframe
justificationData = pd.read_csv ('../data/Justification_Data/Justification_Data.tsv', sep='\t')

# Removed Unwanted Column
del justificationData["Unnamed: 0"]

In [29]:
summarisationDataDict = {}

for json_id in summarisationData.columns:
    
    features = {}
    
    justDataRow = justificationData.loc[justificationData['json_file_id'] == json_id]
    
    features["claim"] = justDataRow["claim"].values[0]
    features["justification_label"] = justDataRow["justification_label"].values[0]
    
    summarisationDataDict[json_id] = features

In [39]:
summarisationData = pd.DataFrame().from_dict(data=summarisationDataDict, orient="index")

summarisationData["justification_label"].value_counts()

distortion    49
unfounded     37
emphasis      35
Name: justification_label, dtype: int64

In [31]:
import random
from sklearn.model_selection import train_test_split

In [32]:
justification_classes = {"distortion": 14, "unfounded": 2, "emphasis": 0}
for just_class in justification_classes:
    
    summarisationDataIndices = list(summarisationData.index[summarisationData['justification_label'] == just_class])
    
    randIndexList = random.sample(summarisationDataIndices, justification_classes[just_class])
    if randIndexList:
        summarisationData.drop(randIndexList, inplace=True)

In [38]:
summarisationData["justification_label"].value_counts()

distortion    35
emphasis      35
unfounded     35
Name: justification_label, dtype: int64

In [34]:
tempData = summarisationData

y = tempData["justification_label"].to_frame()

In [36]:
X_train, X_test, y_train, y_test = train_test_split(tempData, y,stratify=y, test_size=0.3)

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, stratify=y_test, test_size=0.4)

In [37]:
X_train.to_csv('../data/Train_Eval_Test_Data/train2.tsv', sep="\t")
X_test.to_csv('../data/Train_Eval_Test_Data/test2.tsv', sep="\t")
X_val.to_csv('../data/Train_Eval_Test_Data/val2.tsv', sep="\t")